In [17]:
import requests
import re
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from json import JSONDecodeError
from collections import defaultdict
import json
import numpy as np

from parse_dbpedia import *

In [2]:
es = Elasticsearch()
es.info()

{'name': 'BIEBER',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'nN32_N-8T1OLuu3AJOSr2g',
 'version': {'number': '7.9.1',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '083627f112ba94dffc1232e8b42b73492789ef91',
  'build_date': '2020-09-01T21:22:21.964974Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [15]:
def analyze_query(es, index, query, field='body'):
    """Analyzes a query with respect to the relevant index. 
    
    Arguments:
        es: Elasticsearch object instance.
        query: String of query terms.
        field: The field with respect to which the query is analyzed. 
        index: Name of the index with respect to which the query is analyzed.  
    
    Returns:
        A list of query terms that exist in the specified field among the documents in the index. 
    """
    tokens = es.indices.analyze(index=index, body={'text': query})['tokens']
    query_terms = []
    for t in sorted(tokens, key=lambda x: x['position']):
        ## Use a boolean query to find at least one document that contains the term.
        hits = es.search(index=index,
                         body={
                             'query': {
                                 'match': {
                                     field: t['token']
                                 }
                             }
                         },
                         _source=False,
                         size=1).get('hits', {}).get('hits', {})
        doc_id = hits[0]['_id'] if len(hits) > 0 else None
        if doc_id is None:
            continue
        query_terms.append(t['token'])
    return query_terms

def baseline_retrieval(es, index_name, query, k=100):
    """Performs baseline retrival on index.
    
    Arguments:
        index_name: A string of text.
        query: A string of text, space separated terms.
        k: An integer.
        
    Returns:
        A list of entity IDs as strings, up to k of them, in descending order of scores.
    """

    hits = es.search(index=index_name, q=query, _source=True,
                     size=k)['hits']['hits']
    return [(doc['_score'], doc['_source']['types']) for doc in hits]

In [4]:
INDEX_NAME = 'entity_centric_bm25_long_abstract'
INDEX_SETTINGS = {
    'mappings': {
        'properties': {
            'body': {
                'type': 'text',
                'term_vector': 'yes',
                'analyzer': 'english'
            },
            'types': {
                'type': 'text',
                'term_vector': 'yes',
                'analyzer': 'english'
            },
        }
    }
}

In [5]:
def reset_index(es, index):
    if es.indices.exists(INDEX_NAME):
        es.indices.delete(index=INDEX_NAME)

    es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)
    
def get_data(index, doc):
    num_docs = len(doc) // 10
    for i, (doc_id, body) in enumerate(doc.items()):
        yield {'_index': index, '_id': doc_id, '_source': body}
        if i % num_docs == 0:
            print('{}% done'.format((i // num_docs)*10))

In [6]:
ontology = get_ontology_tree()
doc_entity = get_entity_docs()
entity_data = get_entity_data()
for entity, body in doc_entity.items():
    ancestors = []
    for entity_type in body['types'].split():
        ancestors.extend(get_ancestors(ontology, entity_type))
    doc_entity[entity]['types'] = ' '.join(set(ancestors))
    doc_entity[entity]['body'] += ' ' + entity_data.get(entity, '')

Num entities with types:  4767652
Num entities with data:  4935279


In [7]:
doc_entity['http://dbpedia.org/resource/Library_of_Alexandria']

{'body': 'Library of Alexandria The Royal Library of Alexandria or Ancient Library of Alexandria in Alexandria, Egypt, was one of the largest and most significant libraries of the ancient world. It was dedicated to the Muses, the nine goddesses of the arts. It flourished under the patronage of the Ptolemaic dynasty and functioned as a major center of scholarship from its construction in the 3rd century BC until the Roman conquest of Egypt in 30 BC, with collections of works, lecture halls, meeting rooms, and gardens. The library was part of a larger research institution called the Musaeum of Alexandria, where many of the most famous thinkers of the ancient world studied. The library was created by Ptolemy I Soter, who was a Macedonian general and the successor of Alexander the Great. Most of the books were kept as papyrus scrolls. It is unknown precisely how many such scrolls were housed at any given time, but estimates range from 40,000 to 400,000 at its height. Arguably, this library

In [9]:
reset_index(es, INDEX_NAME)
for success, info in parallel_bulk(es,
                                   get_data(INDEX_NAME, doc_entity),
                                   thread_count=12,
                                   chunk_size=5000,
                                   queue_size=6):
    if not success:
        print('A document failed:', info)

0% done
1% done
2% done
3% done
4% done
5% done
6% done
7% done
8% done
9% done
10% done


In [10]:
es.cat.count(INDEX_NAME, params={"format": "json"})

[{'epoch': '1604409920', 'timestamp': '13:25:20', 'count': '4767652'}]

In [11]:
with open('./data/train_set', 'r') as f:
    train_set = json.load(f)

In [25]:
k = 20
for item in train_set:
    if item['category'] == 'resource':
        query = analyze_query(es, INDEX_NAME, item['question'])
        scores = defaultdict(int)
        for score, types in baseline_retrieval(es, INDEX_NAME, ' '.join(query), k):
            for t in types.split():
                scores[t] += score
        top_res = max(scores.items(), key=lambda x: x[1])[0]
        final_types = ['dbo:' + i for i in get_ancestors(ontology, top_res)]
        
        #print(item['question'], query)
        #print('retrieval: ', final_types)
        #print('ground truth: ', item['type'])
        #print()

With whom were the engagements of Alpine Brigade Taurinense? ['whom', 'were', 'engagements', 'alpine', 'brigade', 'taurinense']
retrieval:  ['dbo:Organisation', 'dbo:Agent']
ground truth:  ['dbo:Country', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

What is the first name of John Lothrop Motley, that is the equivalent of Gianni? ['what', 'first', 'name', 'john', 'lothrop', 'motley', 'equivalent', 'gianni']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:GivenName', 'dbo:Name']

What award was John Williams nominated for in Close Encounters of the Third Kind? ['what', 'award', 'john', 'williams', 'nominated', 'close', 'encounters', 'third', 'kind']
retrieval:  ['dbo:Work']
ground truth:  ['dbo:Award']

What is the allegiance of Albert Kwesi Ocran ? ['what', 'allegiance', 'albert', 'kwesi', 'ocran']
retrieval:  ['dbo:TimePeriod']
ground truth:  ['dbo:Country', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

What award did Keanu Reeves receive, which was also won by Sandra Bullo

Is Andres Pastrana Arango the current holder of the presidency in Colombia? ['andres', 'pastrana', 'arango', 'current', 'holder', 'presidency', 'colombia']
retrieval:  ['dbo:Person', 'dbo:Agent']
ground truth:  ['dbo:Person', 'dbo:Agent']

Who is the spouse of the person who painted Glorious victory? ['who', 'spouse', 'person', 'who', 'painted', 'glorious', 'victory']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:Person', 'dbo:Agent']

Name the developer of Fresnel lens, who graduated from Lycee Malherbe. ['name', 'developer', 'fresnel', 'lens', 'who', 'graduated', 'from', 'lycee', 'malherbe']
retrieval:  ['dbo:Animal', 'dbo:Eukaryote', 'dbo:Species']
ground truth:  ['dbo:Person', 'dbo:Agent']

What is the hub and who is the owner of Shandong Airlines? ['what', 'hub', 'who', 'owner', 'shandong', 'airlines']
retrieval:  ['dbo:Airline', 'dbo:Company', 'dbo:Organisation', 'dbo:Agent']
ground truth:  ['dbo:Airport', 'dbo:Infrastructure', 'dbo:ArchitecturalStructure', 'dbo:Place', 'dbo:Loc

What is the Home Stadium of the Kenya national under-20 football team where Matin Suji was also born ? ['what', 'home', 'stadium', 'kenya', 'national', 'under', '20', 'football', 'team', 'where', 'matin', 'suji', 'also', 'born']
retrieval:  ['dbo:SportsTeam', 'dbo:Organisation', 'dbo:Agent']
ground truth:  ['dbo:Country', 'dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

Of what country is Moon Jae-in the leader? ['what', 'country', 'moon', 'jae', 'leader']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:Country', 'dbo:State', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

Tell me about the state in the Confederation of the Rhine located in the administrative territorial entity of Weimar Republic. ['tell', 'me', 'about', 'state', 'confederation', 'rhine', 'located', 'administrative', 'territorial', 'entity', 'weimar', 'republic']
retrieval:  ['dbo:Place']
ground truth:  ['dbo:State', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

Who was in conf

Which department is responsible for the city of Paris ['which', 'department', 'responsible', 'city', 'paris']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:Organisation', 'dbo:Agent']

Where did Abraham Lincoln die? ['where', 'did', 'abraham', 'lincoln', 'die']
retrieval:  ['dbo:Work']
ground truth:  ['dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

Of what country was Jefferson Davis a citizen? ['what', 'country', 'jefferson', 'davis', 'citizen']
retrieval:  ['dbo:Place']
ground truth:  ['dbo:Country', 'dbo:State', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

Politicians born in Bangalore are now in which parties? ['politicians', 'born', 'bangalore', 'now', 'which', 'parties']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:PoliticalParty']

Is Vidhan Sabha executive body of Assam? ['vidhan', 'sabha', 'executive', 'body', 'assam']
retrieval:  ['dbo:Place']
ground truth:  ['dbo:Organisation', 'dbo:Agent']

Which senator served in military units V

Who is the master of the musical score of Songs of sundrie natures? ['who', 'master', 'musical', 'score', 'songs', 'sundrie', 'natures']
retrieval:  ['dbo:Work']
ground truth:  ['dbo:Person', 'dbo:Agent']

Who is the owner of Rolls-Royce? ['who', 'owner', 'rolls', 'royce']
retrieval:  ['dbo:MeanOfTransportation']
ground truth:  ['dbo:Company', 'dbo:Organisation', 'dbo:Agent']

Which company released the software RenderMan? ['which', 'company', 'released', 'software', 'renderman']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:Company', 'dbo:Organisation', 'dbo:Agent']

Who replaced Arthur Fadden's held position as Prime Minister of Australia? ['who', 'replaced', 'arthur', "fadden's", 'held', 'position', 'prime', 'minister', 'australia']
retrieval:  ['dbo:SocietalEvent', 'dbo:Event']
ground truth:  ['dbo:Person', 'dbo:Agent']

Who are currently playing for Barcelona FC? ['who', 'currently', 'playing', 'barcelona', 'fc']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:SoccerPlayer', 'dbo:

...

What city with millions of people has the official language of Urdu? ['what', 'city', 'millions', 'people', 'has', 'official', 'language', 'urdu']
retrieval:  ['dbo:Language']
ground truth:  ['dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

Which royal people are buried in Rome? ['which', 'royal', 'people', 'buried', 'rome']
retrieval:  ['dbo:Place']
ground truth:  ['dbo:Royalty', 'dbo:Person', 'dbo:Agent']

Who is the stock holder of the bank where  Peter Grauer works  ? ['who', 'stock', 'holder', 'bank', 'where', 'peter', 'grauer', 'works']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:Politician', 'dbo:Person', 'dbo:Agent']

What award did Linus Pauling receive? ['what', 'award', 'did', 'linus', 'pauling', 'receive']
retrieval:  ['dbo:Person', 'dbo:Agent']
ground truth:  ['dbo:Award']

What is the craft of the Ralph Flanders which is also the profession of  Claiborne Pell ? ['what', 'craft', 'ralph', 'flanders', 'which', 'also', 'profession', 'claib

Which is the Norse deity that is partner of Odin? ['which', 'norse', 'deity', 'partner', 'odin']
retrieval:  ['dbo:Place']
ground truth:  ['dbo:Deity', 'dbo:Agent']

On the Churchill River what lakes are on the river? ['churchill', 'river', 'what', 'lakes', 'river']
retrieval:  ['dbo:Place']
ground truth:  ['dbo:Lake', 'dbo:BodyOfWater', 'dbo:NaturalPlace', 'dbo:Place', 'dbo:Location']

What sports are played by school which owns Fr. Agnel Stadium ? ['what', 'sports', 'played', 'school', 'which', 'owns', 'fr', 'agnel', 'stadium']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:Sport', 'dbo:Activity']

List the television shows one of whose distributor's divisions is Warner Bros. Animation. ['list', 'television', 'shows', 'one', 'whose', "distributor's", 'divisions', 'warner', 'bros', 'animation']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:TelevisionShow', 'dbo:Work']

Which restaurateur coined the term hot dog? ['which', 'restaurateur', 'coined', 'term', 'hot', 'dog']
retrieval:  ['

What was the language of the single which came before To Know Him Is to Love Him? ['what', 'language', 'single', 'which', 'came', 'before', 'know', 'him', 'love', 'him']
retrieval:  ['dbo:Work']
ground truth:  ['dbo:Country', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

Which academic field for the history of the United States divides into the state of Michigan? ['which', 'academic', 'field', 'history', 'united', 'states', 'divides', 'state', 'michigan']
retrieval:  ['dbo:Place']
ground truth:  ['dbo:Country', 'dbo:State', 'dbo:PopulatedPlace', 'dbo:Place', 'dbo:Location']

In what venue did Divorcing the Late Upper Palaeolithic demographic histories of mtDNA haplogroups M1 and U6 in Africa get a bibliographic citation? ['what', 'venue', 'did', 'divorcing', 'late', 'upper', 'palaeolithic', 'demographic', 'histories', 'mtdna', 'haplogroups', 'm1', 'u6', 'africa', 'get', 'bibliographic', 'citation']
retrieval:  ['dbo:Work']
ground truth:  ['dbo:AcademicJournal', 'dbo:PeriodicalLit

Which leader of United States House of Representatives was the appinter of ohn Drayton ? ['which', 'leader', 'united', 'states', 'house', 'representatives', 'ohn', 'drayton']
retrieval:  ['dbo:SocietalEvent', 'dbo:Event']
ground truth:  ['dbo:OfficeHolder', 'dbo:Person', 'dbo:Agent']

Which is the exhibition history of Le Déjeuner sur l'herbe? ['which', 'exhibition', 'history', 'le', 'sur', "l'herbe"]
retrieval:  ['dbo:Place']
ground truth:  ['dbo:Event']

Which military unit's  garrison is Pentagon and command structure is United States Department of Defense? ['which', 'military', "unit's", 'garrison', 'pentagon', 'command', 'structure', 'united', 'states', 'department', 'defense']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:MilitaryUnit', 'dbo:Organisation', 'dbo:Agent']

Who is Tom Brady's spouse and where were they married? ['who', 'tom', "brady's", 'spouse', 'where', 'were', 'married']
retrieval:  ['dbo:Agent']
ground truth:  ['dbo:Person', 'dbo:Agent']

What is the allegiance 

RequestError: RequestError(400, 'x_content_parse_exception', "[1:9] [analyze_request] text doesn't support values of type: VALUE_NULL")